In [1]:
#| default_exp fasthtml_patching

In [2]:
#| export
from __future__ import annotations

# fasthtml patching
> Monkey-patching some fasthtml features/issues while waiting to submit the corresponding PRs.

# Prologue

In [3]:
#| export
import fastcore.all as FC
from fastcore.xml import to_xml
from fasthtml.components import Div
from fasthtml.core import FT
from fasthtml.jupyter import JupyUvi
from fasthtml.xtend import Script
from IPython.display import display
from IPython.display import HTML


In [4]:
#| export
from bridget.helpers import bridge_cfg


In [5]:
import fasthtml.components as ft
from fastcore.test import *
from fasthtml.components import highlight
from fasthtml.components import show
from fasthtml.components import showtags
from fasthtml.core import FastHTML
from fasthtml.core import fh_cfg
from fasthtml.core import fhjsscr
from fasthtml.core import scopesrc
from fasthtml.core import surrsrc
from httpx import get
from IPython.display import clear_output
from IPython.display import Javascript
from IPython.display import Markdown
from olio.common import setup_console


In [6]:
from fasthtml.components import P, Pre, Text, Span

----


In [7]:
console, cprint = setup_console(140)

----

# FastHTML Bridge display

1. This functionality patches FastHTML's IPython display system
3. This implementation:
   - Overrides all FT IPython MIME methods
   - Uses `bridge_cfg.auto_show` for display opt-in

This is a convenience and not core of this proof-of-concept notebooks.

In [8]:
fh_cfg['auto_id'] = False
bridge_cfg.auto_show=False

In [9]:
d = Div(style='color: red;', hx_trigger='click')(Text('Hi!'))
print(f"d: \n{d}"); display(Markdown('----'))
print(f"showtags(d): \n{showtags(d)}"); display(Markdown('----'))
display(Markdown(highlight(d)+'\n----'))
show(d)

d: 
<div hx-trigger="click" style="color: red;"><text>Hi!</text></div>


----

showtags(d): 
<code><pre>
&lt;div hx-trigger=&quot;click&quot; style=&quot;color: red;&quot;&gt;
&lt;text&gt;Hi!&lt;/text&gt;&lt;/div&gt;

</code></pre>


----

```html
<div hx-trigger="click" style="color: red;">
<text>Hi!</text></div>

```
----

In [10]:
Div("I'm a Div!")

```html
<div>I&#x27;m a Div!</div>

```

In [11]:
#| exporti

@FC.patch
def _repr_mimebundle_(self: FT, include=None, exclude=None):
    mb = {'text/plain': repr(self)}
    if bridge_cfg.auto_show: mb['text/html'] = self.__html__()
    else: mb['text/markdown'] = self._repr_markdown_()
    return mb

In [12]:
# #| exporti

# @FC.patch
# def _ipython_display_(self: FT):
#     IDISPLAY(self._repr_mimebundle_(), raw=True)

In [13]:
Div('me too!')

```html
<div>me too!</div>

```

In [14]:
with bridge_cfg(auto_show=True):
    display(Div("But I'm prettier!"))

div(("But I'm prettier!",),{})

In [15]:
Div('back to tags!')

```html
<div>back to tags!</div>

```

In [16]:
bridge_cfg.auto_show = True
ft.Details(open=True)(ft.Summary('dddd'), ft.Pre('eeee'))

details((summary(('dddd',),{}), pre(('eeee',),{})),{'open': True})

In [17]:
with bridge_cfg(auto_show=False): display(ft.Button('ffff'))
ft.Button('ffff')

```html
<button>ffff</button>
```

button(('ffff',),{})

Revert to FT usual.

In [18]:
del FT._repr_mimebundle_  # type: ignore

In [19]:
Div("I'm a Div!")

```html
<div>I&#x27;m a Div!</div>

```

# FastHTML Jupyter display

Opt-in to display FastHTML objects in the notebook as HTML instead of the default markdown `repr`.

:::{.callout-warning    }
Note recent FastHTML [v0.9.0](https://github.com/AnswerDotAI/fasthtml/releases/tag/0.9.0) has improved Jupyter support substancially and now have this same functionality (with `render_ft`, in v0.9.1 on by default).

Unfortunately, current implementation doesn't work in VSCode/Cursor and probably any other Jupyter-ish environment that sandbox notebook outputs.

In VSCode/Cursor besides that, `render_ft` doesn't work as expected, as Quarto renders markdown in a static shadow-root.
:::


## Patching FastHTML jupyter to work in VSCode

In [20]:
#| exporti

def htmx_config_port(host='localhost', port=8000):
    display(HTML('''
<script>
document.body.addEventListener('htmx:configRequest', (event) => {
    if(event.detail.path.includes('://')) return;
    htmx.config.selfRequestsOnly=false;
    event.detail.path = `http://%s:%s${event.detail.path}`;
});
</script>''' % (host, port)))

In [21]:
#| export

class JupyUviB(JupyUvi):
    "Start and stop a Jupyter compatible uvicorn server with ASGI `app` on `port` with `log_level`"
    def __init__(self, app, log_level="error", host='0.0.0.0', port=8000, start=True, **kwargs):
        self.kwargs = kwargs
        FC.store_attr(but='start')
        self.server = None
        if start: self.start()
        htmx_config_port(host, port)

In [22]:
# type: ignore

app = FastHTML()
rt = app.route

@app.route
def index(): return 'hi'

port = 8000
server = JupyUviB(app, port=port)

In [23]:
get(f'http://localhost:{port}').text

'hi'

In [24]:
fh_cfg['auto_id']=True

In [ ]:
show(Script(src='https://unpkg.com/htmx.org@2.0.4/dist/htmx.js'), fhjsscr, scopesrc, surrsrc)
clear_output()

In [ ]:
display(Javascript('''
if (window.htmx) htmx.process(document.body);
'''))
clear_output()

In [27]:
#| exporti

def render_ft():
    try: del FT._repr_mimebundle_  # type: ignore  # undo our patch to allow FastHTML Jupyter to work as intended
    except Exception: pass
    @FC.patch
    def _repr_html_(self:FT): 
        return to_xml(
            (Div(self), 
            Script('if (window.htmx) htmx.process(document.body)'))
        )

In [28]:
render_ft()

After importing `fasthtml.jupyter` and calling `render_ft()`, FT components render directly in the notebook.

In [29]:
(c := Div('Cogito ergo sum'))

div(('Cogito ergo sum',),{'id': '_vJLxvxx7Sp_yDxxj9T8G4g'})

Handlers are written just like a regular web app:

In [30]:
# type: ignore

@rt
def hoho(): return P('loaded!'), Div('hee hee', id=c, hx_swap_oob='true')

All the usual `hx_*` attributes can be used:

In [31]:
P('not loaded', hx_get=hoho, hx_trigger='load')  # type: ignore

p(('not loaded',),{'hx-get': <fasthtml.core._mk_locfunc.<locals>._lf object at 0x1644f6f90>, 'hx-trigger': 'load', 'id': '_FgmThyaWTqW7x_tWVuojAg'})

FT components can be used directly both as `id` values and as `hx_target` values.

In [32]:
(c := Div(''))

div(('',),{'id': '_LnBkdZFJTIuysLH2XlYB5Q'})

In [33]:
# type: ignore

@rt
def foo(): return Div('foo bar')

P('hi', hx_get=foo, hx_trigger='load', hx_target=c)

p(('hi',),{'hx-get': <fasthtml.core._mk_locfunc.<locals>._lf object at 0x1644f7860>, 'hx-trigger': 'load', 'hx-target': '#_LnBkdZFJTIuysLH2XlYB5Q', 'id': '_LaaYY6XbTG2J4pZI1BHoTg'})

## Stop server
----

In [34]:
server.stop()

----


# Colophon
----


In [35]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [36]:
if FC.IN_NOTEBOOK:
    nb_path = '03_fasthtml_patching.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)